In [2]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import warnings 
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import f_oneway

### Funcion: eval_model

Esta función debe recibir un target, unas predicciones para ese target, un argumento que determine si el problema es de regresión o clasificación y una lista de métricas:
* Si el argumento dice que el problema es de regresión, la lista de métricas debe admitir las siguientes etiquetas RMSE, MAE, MAPE, GRAPH.
* Si el argumento dice que el problema es de clasificación, la lista de métrica debe admitir, ACCURACY, PRECISION, RECALL, CLASS_REPORT, MATRIX, MATRIX_RECALL, MATRIX_PRED, PRECISION_X, RECALL_X. En el caso de las _X, X debe ser una etiqueta de alguna de las clases admitidas en el target.

Funcionamiento:
* Para cada etiqueta en la lista de métricas:
- RMSE, debe printar por pantalla y devolver el RMSE de la predicción contra el target.
- MAE, debe pintar por pantalla y devolver el MAE de la predicción contra el target. 
- MAPE, debe pintar por pantalla y devolver el MAPE de la predcción contra el target. Si el MAPE no se pudiera calcular la función debe avisar lanzando un error con un mensaje aclaratorio
- GRAPH, la función debe pintar una gráfica comparativa (scatter plot) del target con la predicción
- ACCURACY, pintará el accuracy del modelo contra target y lo retornará.
- PRECISION, pintará la precision media contra target y la retornará.
- RECALL, pintará la recall media contra target y la retornará.
- CLASS_REPORT, mostrará el classification report por pantalla.
- MATRIX, mostrará la matriz de confusión con los valores absolutos por casilla.
- MATRIX_RECALL, mostrará la matriz de confusión con los valores normalizados según el recall de cada fila (si usas ConfussionMatrixDisplay esto se consigue con normalize = "true")
- MATRIX_PRED, mostrará la matriz de confusión con los valores normalizados según las predicciones por columna (si usas ConfussionMatrixDisplay esto se consigue con normalize = "pred")
- PRECISION_X, mostrará la precisión para la clase etiquetada con el valor que sustituya a X (ej. PRECISION_0, mostrará la precisión de la clase 0)
- RECALL_X, mostrará el recall para la clase etiquetada co nel valor que sustituya a X (ej. RECALL_red, mostrará el recall de la clase etiquetada como "red")

NOTA1: Como puede que la función devuelva varias métricas, debe hacerlo en una tupla en el orden de aparición de la métrica en la lista que se le pasa como argumento. Ejemplo si la lista de entrada es ["GRAPH","RMSE","MAE"], la fución pintará la comparativa, imprimirá el RMSE y el MAE (da igual que lo haga antes de dibujar la gráfica) y devolverá una tupla con el (RMSE,MAE) por ese orden.
NOTA2: Una lista para clasificación puede contener varias PRECISION_X y RECALL_X, pej ["PRECISION_red","PRECISION_white","RECALL_red"] es una lista válida, tendrá que devolver la precisión de "red", la de "white" y el recall de "red". Si algunas de las etiquetas no existe debe arrojar ese error y detener el funcionamiento.

In [2]:
def eval_model(target, predicciones, tipo_de_problema, metricas):

    """
    Función que evalua un modelo de Machine Learning utilizando diferentes métricas para problemas de regresión o clasificación

    Argumentos:
    target (tipo array): Valores del target
    predicciones (tipo array): Valores predichos por el modelo
    tipo_de_problema (str): Puede ser de regresión o clasificación
    metricas (list): Lista de métricas a calcular:
                     Para problemas de regresión: "RMSE", "MAE", "MAPE", "GRAPH"
                     Para problemas de clasificación: "ACCURACY", "PRECISION", "RECALL", "CLASS_REPORT", "MATRIX", "MATRIX_RECALL", "MATRIX_PRED", "PRECISION_X", "RECALL_X"

    Retorna:
    tupla: Devuelve una tupla con los resultados de las métricas especificadas
    """

    results = []

    # Regresión

    if tipo_de_problema == "regresion":

        for metrica in metricas:
            
            if metrica == "RMSE":
                rmse = np.sqrt(mean_squared_error(target, predicciones))
                print(f"RMSE: {rmse}")
                results.append(rmse)
            
            elif metrica == "MAE":
                mae = mean_absolute_error(target, predicciones)
                print(f"MAE: {mae}")
                results.append(mae)

            elif metrica == "MAPE":
                try:
                    mape = np.mean(np.abs((target - predicciones) / target)) * 100
                    print(f"MAPE: {mape}")
                    results.append(mape)
                except ZeroDivisionError:
                    raise ValueError("No se puede calcular el MAPE cuando hay valores en el target iguales a cero")
           
            elif metrica == "GRAPH":
                plt.scatter(target, predicciones)
                plt.xlabel("Real")
                plt.ylabel("Predicción")
                plt.title("Gráfico de Dispersión: Valores reales VS Valores predichos")
                plt.show()

     # Clasificación
                
    elif tipo_de_problema == "clasificacion":

        for metrica in metricas:
            
            if metrica == "ACCURACY":
                accuracy = accuracy_score(target, predicciones)
                print(f"Accuracy: {accuracy}")
                results.append(accuracy)

            elif metrica == "PRECISION":
                precision = precision_score(target, predicciones, average = "macro")
                print(f"Precision: {precision}")
                results.append(precision)

            elif metrica == "RECALL":
                recall = recall_score(target, predicciones, average = "macro")
                print(f"Recall: {recall}")
                results.append(recall)

            elif metrica == "CLASS_REPORT":
                print("Classification Report:")
                print(classification_report(target, predicciones))

            elif metrica == "MATRIX":
                print("Confusion Matrix (Absolute Values):")
                print(confusion_matrix(target, predicciones))

            elif metrica == "MATRIX_RECALL":
                disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(target, predicciones))
                disp.plot(normalize = "true")
                plt.title("Confusion Matrix (Normalized by Recall)")
                plt.show()

            elif metrica == "MATRIX_PRED":
                disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(target, predicciones))
                disp.plot(normalize = "pred")
                plt.title("Confusion Matrix (Normalized by Prediction)")
                plt.show()

            elif "PRECISION_" in metrica:
                class_label = metrica.split("_")[-1]
                try:
                    precision_class = precision_score(target, predicciones, labels = [class_label])
                    print(f"Precisión para la clase {class_label}: {precision_class}")
                    results.append(precision_class)
                except ValueError:
                    raise ValueError(f"La clase {class_label} no está presente en las predicciones")
                
            elif "RECALL_" in metrica:
                class_label = metrica.split("_")[-1]
                try:
                    recall_class = recall_score(target, predicciones, labels = [class_label])
                    print(f"Recall para la clase {class_label}: {recall_class}")
                    results.append(recall_class)
                except ValueError:
                    raise ValueError(f"La clase {class_label} no está presente en las predicciones")
                
    # Si no es regresión o clasificación

    else:
        raise ValueError("El tipo de problema debe ser de regresión o clasificación")

    return tuple(results)

Función #2

### Funcion: plot_features_num_classification

Esta función recibe un dataframe, una argumento "target_col" con valor por defecto "", una lista de strings ("columns") cuyo valor por defecto es la lista vacía,  y un argumento ("pvalue") con valor 0.05 por defecto.

Si la lista no está vacía, la función pintará una pairplot del dataframe considerando la columna designada por "target_col" y aquellas incluidas en "column" que cumplan el test de ANOVA para el nivel 1-pvalue de significación estadística. La función devolverá los valores de "columns" que cumplan con las condiciones anteriores. Ojo, se espera que las columnas sean numéricas. El pairplot utilizar como argumento de hue el valor de target_col.

Si la lista está vacía, entonces la función igualará "columns" a las variables numéricas del dataframe y se comportará como se describe en el párrafo anterior.

EXTRA_1: Se valorará adicionalmente el hecho de que si el número de valores posibles de target_Col se superior a 5, se usen diferentes pairplot diferentes, en cuyo caso pintará un pairplot por cada 5 valores de target posibles.

EXTRA_2: Se valorará adicionalmente el hecho de que si la lista de columnas a pintar es grande se pinten varios pairplot con un máximo de cinco columnas en cada pairplot (siendo siempre una de ellas la indicada por "target_col")

De igual manera que en la función descrita anteriormente deberá hacer un check de los valores de entrada y comportarse como se describe en el último párrafo de la función `get_features_num_classification`

In [1]:
def plot_features_num_classification(dataframe, target_col="", columns=None, pvalue=0.05):
    """
    Genera pairplots para visualizar la relación entre las columnas numéricas de un dataframe y una columna objetivo, 
    filtrando aquellas columnas que pasan una prueba de ANOVA según un nivel de significación especificado.

    Argumentos:
    dataframe (pd.DataFrame): El dataframe que contiene los datos.
    target_col (str): Nombre de la columna objetivo para la clasificación. Valor por defecto es una cadena vacía.
    columns (list): Lista de nombres de columnas a considerar. Si no se proporciona, se consideran todas las columnas numéricas. Valor por defecto es None.
    pvalue (float): Nivel de significación para la prueba de ANOVA. Valor por defecto es 0.05.

    Retorna:
    list: Devuelve una lista de nombres de columnas que cumplen con el criterio de significación especificado.
    """
    # Validar entradas
    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("dataframe debe ser un DataFrame de pandas")
    if not isinstance(target_col, str):
        raise ValueError("target_col debe ser un string")
    if columns is not None and not all(isinstance(col, str) for col in columns):
        raise ValueError("columns debe ser una lista de strings")
    if not isinstance(pvalue, (int, float)) or not (0 < pvalue < 1):
        raise ValueError("pvalue debe ser un número entre 0 y 1")
    
    # Si columns es None, igualar a las columnas numéricas del dataframe
    if columns is None:
        columns = dataframe.select_dtypes(include=['number']).columns.tolist()
    else:
        # Filtrar solo las columnas numéricas que están en la lista
        columns = [col for col in columns if dataframe[col].dtype in ['float64', 'int64']]
    
    # Asegurarse de que target_col esté en el dataframe
    if target_col and target_col not in dataframe.columns:
        raise ValueError(f"{target_col} no está en el dataframe")
    
    # Filtrar columnas que cumplen el test de ANOVA
    valid_columns = []
    if target_col:
        unique_classes = dataframe[target_col].unique()
        for col in columns:
            groups = [dataframe[dataframe[target_col] == cls][col].dropna() for cls in unique_classes]
            if len(groups) > 1 and all(len(group) > 0 for group in groups):
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    f_val, p_val = f_oneway(*groups)
                if p_val < pvalue:
                    valid_columns.append(col)
    else:
        valid_columns = columns

    # Si no hay columnas válidas, retornar una lista vacía
    if not valid_columns:
        return []

    # Configuración de estilos
    sns.set(style="whitegrid", palette="muted", color_codes=True)
    
    # Crear pairplots agrupados
    max_cols_per_plot = 5  # Máximo de columnas por plot
    if target_col:
        for i in range(0, len(valid_columns), max_cols_per_plot):
            plot_columns = valid_columns[i:i+max_cols_per_plot]
            if target_col not in plot_columns:
                plot_columns.append(target_col)
            
            # Depuración: imprimir columnas y verificar contenido de subset_df
            print(f"Plot columns: {plot_columns}")
            print(f"DataFrame shape: {dataframe.shape}")
            print(f"DataFrame columns: {dataframe.columns.tolist()}")
            
            g = sns.PairGrid(dataframe[plot_columns], hue=target_col)
            g.map_diag(sns.histplot, kde=True, edgecolor='black', linewidth=0.5)
            g.map_offdiag(sns.scatterplot, s=10, edgecolor="w", linewidth=0.5)
            g.add_legend()
            plt.show()
    else:
        # Sin target_col, dividir en grupos de max_cols_per_plot
        for i in range(0, len(valid_columns), max_cols_per_plot):
            plot_columns = valid_columns[i:i+max_cols_per_plot]
            
            # Depuración: imprimir columnas y verificar contenido de subset_df
            print(f"Plot columns: {plot_columns}")
            print(f"DataFrame shape: {dataframe.shape}")
            print(f"DataFrame columns: {dataframe.columns.tolist()}")

            g = sns.PairGrid(dataframe[plot_columns])
            g.map_diag(sns.histplot, kde=True, edgecolor='black', linewidth=0.5)
            g.map_offdiag(sns.scatterplot, s=10, edgecolor="w", linewidth=0.5)
            plt.show()
    
    return valid_columns